In [5]:
import pandas as pd
import json

In [6]:
file = '/data3/greatxue/llm_uncer/json_result/results-bookqa-gpt4/gpt4-bookqa-allin.json'
df = pd.read_json(file)
df.to_csv('bookqa.csv', index=False)
print("=========Saved to .csv file==============")

=========Saved to .csv file==============


In [10]:
with open(file, 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)
df = df.set_index('total')
df

,prompt,prompt_c,prompt_cot,prompt_few,model_answer,model_answer_c,model_answer_cot,model_answer_few,mark,mark_c,...,mark_few,ans,ans_c,ans_cot,ans_few,reference_answer,len,len_c,len_cot,len_few
total,,,,,,,,,,,,,,,,,,,,,
0,Evidence: \n1. **Habitat Depth**: Both frilled...,Question: Frilled sharks and angler fish live ...,Question: Frilled sharks and angler fish live ...,Here are some examples to show you the answer ...,"Based on the provided evidence, it is clear th...","First, let's break down the question and optio...",Evidence:\n1. Frilled sharks and angler fish a...,"To answer the question, let's analyze the choi...",1,1,...,1,A,A,A,A,A,395,715,1117,304
1,Evidence: \n1. **Understanding Gases and Liqui...,Question: Gas can fill any container it is giv...,Question: Gas can fill any container it is giv...,Here are some examples to show you the answer ...,Considering the evidence and analysis provided...,"To answer the question, let's analyze each opt...","First, let's break down the properties of gas ...","To answer this question, consider what we know...",1,1,...,1,D,D,D,D,D,1031,1042,1111,665
2,"Evidence: \nNow, if someone is walking in the ...",Question: When birds migrate south for the win...,Question: When birds migrate south for the win...,Here are some examples to show you the answer ...,The evidence outlines the process of determini...,"First, consider the factual basis for bird mig...","To determine the correct answer, let's careful...",Question: When food is not stored properly in ...,1,1,...,0,A,A,A,C,A,768,826,1422,159
3,Evidence: \nFor each option:\n- Drinking a col...,Question: If a person walks in the opposite di...,Question: If a person walks in the opposite di...,Here are some examples to show you the answer ...,To correctly determine the direction indicated...,To determine the direction a person is walking...,Evidence:\n1. A compass arrow typically points...,"First, let's break down what a compass arrow s...",1,1,...,1,D,D,D,D,D,363,574,506,247
4,Evidence: \n1. **Mammals**: These are complex ...,Question: An example of lots kinetic energy wo...,Question: An example of lots kinetic energy wo...,Here are some examples to show you the answer ...,The question about kinetic energy is somewhat ...,Let's analyze each option step by step to iden...,To determine which scenario involves a lot of ...,Question: When living things break down food\n...,1,1,...,0,D,D,D,X,D,755,718,1264,387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Evidence: \nA. The sun goes to take a rest in ...,"Question: Some fish species, when hunting, wil...",Question: Sea turtles share space with\nOption...,Here are some examples to show you the answer ...,The final answer is: B.,Let's analyze each option step by step:\n\nA. ...,"Alright, let's go step by step.\n\n1. **Unders...",To decide which plant will absorb more sunligh...,1,0,...,1,B,B,C,C,C,23,848,1112,716
196,Evidence: \n1. **A thriving jungle canopy**: C...,"Question: if a person had one of these, they c...",Question: More sunlight will be absorbed by\nO...,Here are some examples to show you the answer ...,The final answer is: A.,"To live permanently underwater, a person would...",To determine which plant will absorb more sunl...,"Question: Some fish species, when hunting, wil...",1,0,...,0,A,D,C,B,C,23,708,1354,338
197,Evidence: \n1. **A very long nose**: Consider ...,Question: Storms lead to\nOptions:\nA. an incr...,"Question: Some fish species, when hunting, wil...",Here are some examples to show you the answer ...,The evidence provided indirectly suggests that...,"To address this question, let's consider the i...",Evidence Analysis:\n\n1. **Training in a schoo...,Question: Which of these is NOT a source of li...,1,0,...,0,A,D,B,X,B,1332,889,1333,357


In [12]:
df_ = df[['mark', 'len', 'ans', 'reference_answer']]
df_c = df[['mark_c', 'len_c', 'ans_c', 'reference_answer']]
df_cot = df[['mark_cot', 'len_cot', 'ans_cot', 'reference_answer']]
df_few = df[['mark_few', 'len_few', 'ans_few', 'reference_answer']]

In [13]:
df_['len_bin'] = pd.qcut(df['len'], q=5)
print(df_['len_bin'].value_counts())
df_

len_bin
(22.999, 340.0]    40
(340.0, 524.0]     40
(524.0, 675.8]     40
(675.8, 845.8]     40
(845.8, 1360.0]    40
Name: count, dtype: int64


/tmp/ipykernel_783539/3773912656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['len_bin'] = pd.qcut(df['len'], q=5)


,mark,len,ans,reference_answer,len_bin
total,,,,,
0,1,395,A,A,"(340.0, 524.0]"
1,1,1031,D,D,"(845.8, 1360.0]"
2,1,768,A,A,"(675.8, 845.8]"
3,1,363,D,D,"(340.0, 524.0]"
4,1,755,D,D,"(675.8, 845.8]"
...,...,...,...,...,...
195,1,23,B,C,"(22.999, 340.0]"
196,1,23,A,C,"(22.999, 340.0]"
197,1,1332,A,B,"(845.8, 1360.0]"


In [17]:
def calculate_accuracy_by_bin(df, bin_column, mark_column):
    # 统计每个 bin 中的总数和 mark=1 的数量
    bin_group = df.groupby(bin_column)[mark_column].agg(['sum', 'count'])
    
    # 计算正确率：mark=1 的数量 / 总数 * 100
    bin_group['accuracy'] = (bin_group['sum'] / bin_group['count']) * 100
    
    return bin_group[['accuracy']]

# 先按 len 分成 10 个 bin 并计算 df_ 的准确率
df_['len_bin'] = pd.qcut(df_['len'], 10)  
accuracy_by_bin_df_ = calculate_accuracy_by_bin(df_, 'len_bin', 'mark')

# 同样对 df_c, df_cot, df_few 执行相同操作
df_c['len_bin'] = pd.qcut(df_c['len_c'], 10)
accuracy_by_bin_df_c = calculate_accuracy_by_bin(df_c, 'len_bin', 'mark_c')

df_cot['len_bin'] = pd.qcut(df_cot['len_cot'], 10)
accuracy_by_bin_df_cot = calculate_accuracy_by_bin(df_cot, 'len_bin', 'mark_cot')

df_few['len_bin'] = pd.qcut(df_few['len_few'], 10)
accuracy_by_bin_df_few = calculate_accuracy_by_bin(df_few, 'len_bin', 'mark_few')

# 将结果输出到控制台
print("Accuracy by Bin for DF_")
print(accuracy_by_bin_df_)

print("\nAccuracy by Bin for DF_C")
print(accuracy_by_bin_df_c)

print("\nAccuracy by Bin for DF_COT")
print(accuracy_by_bin_df_cot)

print("\nAccuracy by Bin for DF_FEW")
print(accuracy_by_bin_df_few)

Accuracy by Bin for DF_
                 accuracy
len_bin                  
(22.999, 265.8]     100.0
(265.8, 340.0]      100.0
(340.0, 436.1]      100.0
(436.1, 524.0]       95.0
(524.0, 610.5]       95.0
(610.5, 675.8]       90.0
(675.8, 757.8]      100.0
(757.8, 845.8]       85.0
(845.8, 965.3]       85.0
(965.3, 1360.0]      75.0

Accuracy by Bin for DF_C
                   accuracy
len_bin                    
(281.999, 606.5]  30.000000
(606.5, 692.8]    35.000000
(692.8, 742.0]    23.809524
(742.0, 814.0]    35.000000
(814.0, 858.5]    31.578947
(858.5, 907.2]    20.000000
(907.2, 969.3]    10.000000
(969.3, 1043.0]   25.000000
(1043.0, 1121.1]  15.000000
(1121.1, 1530.0]  25.000000

Accuracy by Bin for DF_COT
                    accuracy
len_bin                     
(505.999, 848.0]   95.000000
(848.0, 958.4]    100.000000
(958.4, 1016.6]    95.000000
(1016.6, 1083.0]   95.238095
(1083.0, 1130.0]  100.000000
(1130.0, 1225.2]   90.000000
(1225.2, 1303.6]   95.000000
(1303.6, 1436

/tmp/ipykernel_783539/843743830.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['len_bin'] = pd.qcut(df_['len'], 10)
/tmp/ipykernel_783539/843743830.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bin_group = df.groupby(bin_column)[mark_column].agg(['sum', 'count'])
/tmp/ipykernel_783539/843743830.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in